# Results - Eligible CIViC variants for the CIViC smMIPs capture panel

## Tools 

In [1]:
#!/usr/bin/env python3
import json
import numpy as np
import requests
import sys
import pandas as pd
import matplotlib as plt

# Pull in Input Files

In [2]:
##Pull in CIViC Data from JSON
variants_DNA = requests.get('https://civic.genome.wustl.edu/api/panels/DNA-based/qualifying_variants?minimum_score=0').json()['records'] #Call eligible variants

In [3]:
##Pull in smMIPs Final Design
smmips_design_final = pd.read_csv('../data/smmips_panel/final_civic_smmips_with_removed.txt', sep='\t')

In [4]:
##Pull in Failed Variants
failed = pd.read_csv('../data/smmips_panel/failed_probes.txt', sep='\t')

# Evaluate smMIPs Panel

In [6]:
print('Length of Initial CIViC Probes Design: ', len(smmips_design_final) + len(failed))
print('Total Number of Failed Probes: ', len(failed))
print('Length of Final CIViC Probes Design: ', len(smmips_design_final))

Length of Initial CIViC Probes Design:  2124
Total Number of Failed Probes:  70
Length of Final CIViC Probes Design:  2054


In [15]:
##Determine total variants and genes in final panel
genes = []
variants = []
for item in smmips_design_final['mip_name'].drop_duplicates():
    gene = item.split('_')[0]
    if gene not in genes:
        genes.append(gene)
    for variant in item.split('_')[1:-1]:
        if variant == 'N131P (c.390':
            variants.append([gene, 'N131P (c.390_391delTA)'])
        elif variant == '391delTA)':
            continue
        else:
            variants.append([gene, variant])

# Evaluate Failed Probes

In [16]:
##Determine total variants and genes in final panel
genes_failed = []
variants_failed = []
for item in failed['mip name'].drop_duplicates():
    gene = item.split('_')[0]
    if gene not in genes_failed:
        genes_failed.append(gene)
    for variant in item.split('_')[1:-1]:
        if variant == 'N131P (c.390':
            variants_failed.append([gene, 'N131P (c.390_391delTA)'])
        elif variant == '391delTA)':
            continue
        else:
            variants_failed.append([gene, variant])

In [17]:
print('Variants affected by failed probes: ', len(pd.DataFrame(variants_failed).drop_duplicates()))

Variants affected by failed probes:  32


In [18]:
print( 'Genes affected by failed probes: ', len(genes_failed))

Genes affected by failed probes:  16


# Final Genes and Variants in Panel

In [19]:
print('Total Number of Variants in Panel: ', len(pd.DataFrame(variants).drop_duplicates()))
print('Total Number of Genes in Panel: ', len(genes))

Total Number of Variants in Panel:  111
Total Number of Genes in Panel:  65


In [ ]:
#CREATE SUPPLEMENTARY TABLE 1


# Determine Buckets for Evaluation

In [43]:
bucket_directory = {}
variants_df = pd.DataFrame(variants).drop_duplicates()
variants_df.columns = ['gene', 'variant']
for i,row in variants_df.iterrows():
    gene = row['gene']
    variant = row['variant']
    code = str(gene + '_' + variant)
    
    for item in variants_DNA:
        civic_gene = item['entrez_name']
        civic_variant = item['name']
        if gene == civic_gene and variant == civic_variant:
            start = item['coordinates']['start']
            stop = item['coordinates']['stop']
            length = stop - start
            if length <= 200:
                bucket_directory[code] = 'hotspot_targeting'
            if length > 200:
                if civic_variant == 'AMPLIFICATION' or civic_variant == 'LOSS' or civic_variant == 'DNA BINDING DOMAIN MUTATION':
                    bucket_directory[code] = 'sparse_tiling'
                else:
                    bucket_directory[code] = 'full_tiling'
                

In [44]:
buckets = pd.DataFrame.from_dict(bucket_directory, orient='index').reset_index()
buckets.columns = ['gene_variant', 'bucket']

In [45]:
print('Total Number of Variants in Panel: ', len(pd.DataFrame(variants).drop_duplicates()))
print()
for i,item in pd.DataFrame(buckets.groupby('bucket').size()).iterrows():
    print('Number of variants requiring', i, ':',item[0])

Total Number of Variants in Panel:  111

Number of variants requiring full_tiling : 26
Number of variants requiring hotspot_targeting : 71
Number of variants requiring sparse_tiling : 14


# Determine Clinical Information

In [46]:
clinical_directory = {}
variants_df = pd.DataFrame(variants).drop_duplicates()
variants_df.columns = ['gene', 'variant']
for i,row in variants_df.iterrows():
    gene = row['gene']
    variant = row['variant']
    code = str(gene + '_' + variant)
    clinical_directory[code] = [0,0,0,0]
    for item in variants_DNA:
        civic_gene = item['entrez_name']
        civic_variant = item['name']
        
        if gene == civic_gene and variant == civic_variant:
            for evidence_item in item['evidence_items']:
                if evidence_item['evidence_type'] == 'Predictive':
                    clinical_directory[code][0] += 1
                if evidence_item['evidence_type'] == 'Prognostic':
                    clinical_directory[code][1] += 1
                if evidence_item['evidence_type'] == 'Diagnostic':
                    clinical_directory[code][2] += 1
                if evidence_item['evidence_type'] == 'Predisposing':
                    clinical_directory[code][3] += 1

In [47]:
smmips_evidence_items = pd.DataFrame.from_dict(clinical_directory, orient='index')                       
smmips_evidence_items.columns=['Predictive', 'Prognostic', 'Diagnostic', 'Predisposing']
print('Total Number of Evidence Items: ', smmips_evidence_items.sum().sum(), '\n')
print(smmips_evidence_items.sum(), '\n')
print(smmips_evidence_items.sum()/smmips_evidence_items.sum().sum() * 100, '\n')

Total Number of Evidence Items:  1151 

Predictive      810
Prognostic      226
Diagnostic       52
Predisposing     63
dtype: int64 

Predictive      70.373588
Prognostic      19.635100
Diagnostic       4.517811
Predisposing     5.473501
dtype: float64 



In [311]:
smmips_evidence_items_merged = smmips_evidence_items.reset_index()
smmips_evidence_items_merged.columns = ['gene_variant','Predictive','Prognostic','Diagnostic','Predisposing']
smmips_evidence_items_merged = smmips_evidence_items_merged.merge(buckets, on='gene_variant')

In [318]:
smmips_evidence_items_merged.groupby(['bucket']).sum()

,Predictive,Prognostic,Diagnostic,Predisposing
bucket,,,,
full_tile,116,64,4,0
limited,544,146,48,48
sparse_tile,150,16,0,1


In [ ]:
#CREATE TABLE 1


# Evaluate Total CIViC Variants Covered by CIViC Panel

In [48]:
bed_smmips = pd.read_csv('../data/smmips_panel/smmips_coordinates_sorted_merged.bed.txt', sep='\t', header=None)
bed_smmips.columns = ['chr', 'start', 'stop']
bed_smmips['start'] = pd.to_numeric(bed_smmips['start'])
bed_smmips['stop'] = pd.to_numeric(bed_smmips['stop'])
bed_smmips['chr'] = bed_smmips['chr'].astype(str)

In [51]:
length_bed = 0
for i,row in bed_smmips.iterrows():
    if row['stop'] - row['start'] < 200:
        length_bed += row['stop'] - row['start']
    else:
        length_bed += 110
print('Length of panel: ', length_bed/1000, 'kb')

Length of panel:  61.544 kb


In [52]:
genes_all = []
variants_all = []
for item in variants_DNA:
    chrom = str(item['coordinates']['chromosome'])
    start = int(item['coordinates']['start'])
    stop = int(item['coordinates']['stop'])
    for i,row in bed_smmips.iterrows():
        if chrom == row[0] and start > row[1] and stop < row[2]:
            gene = item['entrez_name']
            variant = item['entrez_name'] + '_' + item['name']
            if gene not in genes_all:
                genes_all.append(gene)
            if variant not in variants_all:
                variants_all.append(variant)

In [53]:
print('Variants Regardless of Actionability Score:' + '\t' + '\n')
print('Total Number of Genes in Panel: ', len(genes_all))
print('Total Number of Variants in Panel: ', len(pd.DataFrame(variants_all).drop_duplicates()))

Variants Regardless of Actionability Score:	

Total Number of Genes in Panel:  49
Total Number of Variants in Panel:  404


In [54]:
variants_all_df = pd.DataFrame(variants_all)
variants_all = []
for i,item in variants_all_df.iterrows():
    gene = item[0].split('_')[0]
    variant = item[0].split('_')[1:]
    variants_all.append([gene, variant])

In [55]:
variants_all_df = pd.DataFrame(variants_all)
variants_all_df.columns = ['gene', 'variant']

In [56]:
variants_all_df.groupby('gene').size()

gene
ABCB1       1
ABL1        7
AKT1        1
ALK        14
ASXL1       1
ATM         2
BRAF       24
BRCA1       1
BRCA2       1
BTK         1
CALR        1
CDK4        1
CTNNB1      3
DNMT3A      1
DPYD        2
EGFR       23
ERBB2       7
ERCC2       1
EZH2        1
FCGR2A      1
FCGR3A      1
FGFR2       1
FLT3        7
GNAS        1
HOXB13      1
IDH1        5
IDH2        3
JAK2        1
KIT        15
KRAS       17
MAP2K7      1
MET         1
MTHFR       1
MYD88       1
NOTCH1      5
NPM1        2
NRAS        7
NT5C2       2
PDGFRA      8
PIK3CA      7
PTEN        4
RET         1
ROS1        3
TP53       10
U2AF1       2
UGT1A1      1
VHL       200
WT1         1
XRCC1       1
dtype: int64